In [1]:
import os
import cv2
from tqdm import tqdm
import json

anno_path = "E:/Datasets/DocBank/DocBank_500K_txt"
output_annotation_file = "E:/Datasets/DocBank/val.json"

In [15]:
coco_data = {}
files = os.listdir(anno_path)    #get all files in the folder
cats = []

for i in tqdm(range(len(files[:1000]))):
    #read the file contents
    fname = anno_path + "/" + files[i]
    f = open(fname, 'r', encoding="utf8")
    lines = f.readlines()        
    f.close()   

    #loop to form categories for the list dictionary
    for j in range(len(lines)):
        line = lines[j].split('	')
        cat_name = line[-1][:-1]
        if cat_name not in cats:            
            cats.append(cat_name)
cats = sorted(cats)
print(cats)

list_dict = []
for i in range(len(cats)):
    tmp = {}
    tmp["supercategory"] = "none"
    tmp["id"] = i
    tmp["name"] = cats[i]
    list_dict.append(tmp)

coco_data["categories"] = list_dict
print(coco_data["categories"])           

100%|█████████████████████████████████████| 1000/1000 [00:01<00:00, 735.58it/s]

['abstract', 'author', 'caption', 'date', 'equation', 'figure', 'footer', 'list', 'paragraph', 'reference', 'section', 'table', 'title']
[{'supercategory': 'none', 'id': 0, 'name': 'abstract'}, {'supercategory': 'none', 'id': 1, 'name': 'author'}, {'supercategory': 'none', 'id': 2, 'name': 'caption'}, {'supercategory': 'none', 'id': 3, 'name': 'date'}, {'supercategory': 'none', 'id': 4, 'name': 'equation'}, {'supercategory': 'none', 'id': 5, 'name': 'figure'}, {'supercategory': 'none', 'id': 6, 'name': 'footer'}, {'supercategory': 'none', 'id': 7, 'name': 'list'}, {'supercategory': 'none', 'id': 8, 'name': 'paragraph'}, {'supercategory': 'none', 'id': 9, 'name': 'reference'}, {'supercategory': 'none', 'id': 10, 'name': 'section'}, {'supercategory': 'none', 'id': 11, 'name': 'table'}, {'supercategory': 'none', 'id': 12, 'name': 'title'}]


In [4]:
coco_data["images"] = []
coco_data["annotations"] = []

img_id = 0
ann_id = 0

for img_id in tqdm(range(len(files))):
    
    #read the file contents
    fname = anno_path + "/" + files[img_id]
    f = open(fname, 'r')
    next(f)    #skip the first row
    lines = f.readlines()        
    f.close()   
    
    
    fname = lines[0][20:28]   #get the name of file without extension
    img_name = fname + ".bmp"
    

    img = img_path + '/' + img_name
    if(not os.path.isfile(img)):
        print('error reading file\n',img)
        continue
    img = cv2.imread(img, 1)
    h, w, c = img.shape

    images_tmp = {}
    images_tmp["file_name"] = img_name
    images_tmp["height"] = h
    images_tmp["width"] = w
    images_tmp["id"] = img_id
    coco_data["images"].append(images_tmp)
    

    j = 1
    while j in range(len(lines)-1):
        
        start = lines[j].find("<") + 1
        end = lines[j].find("Region")
        cat_name = lines[j][start:end]
            
        j += 1
        start = lines[j].find('"') + 1
        end = lines[j].rfind('"')
        coords = lines[j][start:end].split(' ')
        x1, y1 = [int(x) for x in coords[0].split(",")]
        x2, y2 = [int(x) for x in coords[3].split(",")]
        label = cat_name
        

        annotations_tmp = {}
        annotations_tmp["id"] = ann_id
        ann_id += 1
        annotations_tmp["image_id"] = img_id
        annotations_tmp["segmentation"] = []
        annotations_tmp["ignore"] = 0
        annotations_tmp["area"] = (x2-x1)*(y2-y1)
        annotations_tmp["iscrowd"] = 0
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
        annotations_tmp["category_id"] = cats.index(label)

        coco_data["annotations"].append(annotations_tmp)
        j += 2


100%|██████████████████████████████████████| 1600/1600 [01:06<00:00, 24.22it/s]


In [6]:
outfile =  open(output_annotation_file, 'w')
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()